# Big Data project A.Y. 2024-2025

## Members

- Giovanni Antonioni
- Luca Rubboli - 0001083742

## Second job

In [7]:
import org.apache.spark
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import org.apache.spark.sql.SparkSession

import org.apache.spark
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import org.apache.spark.sql.SparkSession


In [7]:
// DO
NOT
EXECUTE - this is needed
just
to
avoid
showing
errors in the
following
cells
sc = spark.SparkContext.getOrCreate()

sc: org.apache.spark.SparkContext = org.apache.spark.SparkContext@595c127b


In [2]:
val
spark = SparkSession.builder
.appName("First job")
.getOrCreate()

spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@2b3ffbe9


# Load Dataset
Define a custom function for loading the parquet files

In [3]:
import org.apache.spark.sql.DataFrame


def dfFromParquetFolder(folderPath: String): DataFrame = spark.read

.option("recursiveFileLookup", "true")
.parquet(folderPath)


import org.apache.spark.sql.DataFrame
dfFromParquetFolder: (folderPath: String)org.apache.spark.sql.DataFrame


We set here the path to the files

In [4]:
val
datasetFolder = "../../../dataset/"
val
yellowCab = s
"$datasetFolder/yellow_cab/"
val
greenCab = s
"$datasetFolder/green_cab/"


datasetFolder: String = ../../../dataset/
yellowCab: String = ../../../dataset//yellow_cab/
greenCab: String = ../../../dataset//green_cab/


# Cleanup

Now we start to cleanup the various dataset that are present in our project.

In [5]:
val
dfYellowCab = dfFromParquetFolder(yellowCab)
val
dfDroppedNa = dfYellowCab.na.drop()

dfYellowCab: org.apache.spark.sql.DataFrame = [VendorID: int, tpep_pickup_datetime: timestamp_ntz ... 17 more fields]
dfDroppedNa: org.apache.spark.sql.DataFrame = [VendorID: int, tpep_pickup_datetime: timestamp_ntz ... 17 more fields]


In [ ]:

val
dfFinal = dfDroppedNa.withColumn("VendorID", dfDroppedNa("VendorID").cast("int"))
dfFinal.count()

[848.322s][warning][gc,alloc] Executor task launch worker for task 6.0 in stage 2.0 (TID 24): Retried waiting for GCLocker too often allocating 121115 words
[848.328s][warning][gc,alloc] Executor task launch worker for task 10.0 in stage 2.0 (TID 28): Retried waiting for GCLocker too often allocating 131221 words
[848.334s][warning][gc,alloc] Executor task launch worker for task 11.0 in stage 2.0 (TID 29): Retried waiting for GCLocker too often allocating 12122 words
[848.335s][warning][gc,alloc] Executor task launch worker for task 1.0 in stage 2.0 (TID 19): Retried waiting for GCLocker too often allocating 10292 words
